# Big Data and Data Analytics for Managers 

In [ ]:
Corona_NLP_BDDA.ipynb

## Merlin John - 015027

AIM- Perform Text Classification on Coronavirus tweets using Py spark


Columns:

1) Location
2) Tweet At
3) Original Tweet
4) Label

In [ ]:
#!pip install pyspark

In [ ]:
#import os       #importing os to set environment variable
#def install_java():
#  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
#  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
#  !java -version       #check java version
#install_java()

In [1]:
#from pyspark.sql import SparkSession
from pyspark.sql.functions import length,  regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer,HashingTF
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline

In [ ]:
#spark = SparkSession.builder \
#        .config("spark.driver.memory","6G")\
#        .config("spark.driver.maxResultSize", "4G") \
#        .getOrCreate()

In [ ]:
#from google.colab import drive
#drive.mount('/gdrive')

In [ ]:
#path = "/gdrive/MyDrive/Colab_data_files/corona_nlp/Corona_NLP_test.csv"
#data=spark.read.csv(path, header = True, inferSchema=True)

In [ ]:
! hdfs dfs -mkdir -p /user/ashok/data_files/corona
! hdfs dfs -put /home/ashok/Documents/corona_nlp_test.csv  /user/ashok/data_files/corona
! hdfs dfs -ls /user/ashok/data_files/corona

In [2]:
! hdfs dfs -mkdir -p /user/ashok/data_files/corona
! hdfs dfs -put /home/ashok/Downloads/corona_nlp_test.csv  /user/ashok/data_files/corona
! hdfs dfs -ls /user/ashok/data_files/corona

put: `/user/ashok/data_files/corona/corona_nlp_test.csv': File exists
Found 1 items
-rw-r--r--   1 ashok supergroup    1002494 2021-11-12 14:41 /user/ashok/data_files/corona/corona_nlp_test.csv


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [224]:
path = "/user/ashok/data_files/corona/corona_nlp_test.csv"
data=spark.read.csv(path, header = True, multiLine = True, inferSchema=True)

In [225]:
data.show()
data.printSchema()

+--------------------+--------------------+--------------------+----------+--------------------+--------------------+
|            UserName|          ScreenName|            Location|   TweetAt|       OriginalTweet|           Sentiment|
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+
|                   1|               44953|                 NYC|02-03-2020|TRENDING: New Yor...|  Extremely Negative|
|                   2|               44954|         Seattle, WA|02-03-2020|When I couldn't f...|            Positive|
|                   3|               44955|                null|02-03-2020|Find out how you ...|  Extremely Positive|
|                   4|               44956|         Chicagoland|02-03-2020|#Panic buying hit...|            Negative|
|                   5|               44957| Melbourne, Victoria|03-03-2020|#toiletpaper #dun...|             Neutral|
|                   6|               44958|         Los 

In [226]:
data.describe().show()

+-------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+
|summary|            UserName|          ScreenName|            Location|   TweetAt|       OriginalTweet|           Sentiment|
+-------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+
|  count|                3980|                3871|                2972|      3799|                3798|                3756|
|   mean|              1899.5|             46851.5|      18701.33203125|      null|                null|                null|
| stddev|  1096.5324892587541|  1096.5324892587541|   35944.79054752188|      null|                null|                null|
| N95 respirators ...|                    |02-03-2020|  We ve got sever...| ""buy tinned foo...|
|    max|She hoped to sta...|            Positive|ÜT: 44.881667,-93...|  Negative| Distance Learni...|            Positive|
+-------+------------

# Data Prepration

In [227]:
result_df = data.groupBy("Location").count().sort("Location", ascending=False)
result_df.show(10)

+--------------------+-----+
|            Location|count|
+--------------------+-----+
|ÜT: 44.881667,-93...|    1|
|ÜT: 43.661815,-79...|    1|
|ÜT: 43.64624,-79....|    1|
|ÜT: 40.725815,-74...|    1|
|ÜT: 40.5896566,-7...|    1|
|ÜT: 3.099457,101....|    1|
|ÜT: 28.508488,-81...|    1|
| ~ JHB, South Africa|    1|
|    www.303Gonzo.com|    1|
|       world citizen|    1|
+--------------------+-----+
only showing top 10 rows



In [228]:
data=data.withColumn('length', length(data['OriginalTweet']))
data.show(10)

+--------------------+--------------------+-------------------+----------+--------------------+------------------+------+
|            UserName|          ScreenName|           Location|   TweetAt|       OriginalTweet|         Sentiment|length|
+--------------------+--------------------+-------------------+----------+--------------------+------------------+------+
|                   1|               44953|                NYC|02-03-2020|TRENDING: New Yor...|Extremely Negative|   228|
|                   2|               44954|        Seattle, WA|02-03-2020|When I couldn't f...|          Positive|   193|
|                   3|               44955|               null|02-03-2020|Find out how you ...|Extremely Positive|    73|
|                   4|               44956|        Chicagoland|02-03-2020|#Panic buying hit...|          Negative|   313|
|                   5|               44957|Melbourne, Victoria|03-03-2020|#toiletpaper #dun...|           Neutral|   252|
|                   6|  

In [229]:
data=data.drop("UserName","ScreenName","Location","TweetAt")
data.filter(data["Sentiment"]=="Negative").show(10)

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|#Panic buying hit...| Negative|   313|
|#CoronaVirus #COV...| Negative|   284|
|When your usual g...| Negative|   201|
|Coronavirus panic...| Negative|   285|
|While you are sto...| Negative|   250|
|Two important pho...| Negative|   271|
|Fellow Uni instru...| Negative|   274|
|1/25 Part 1 - Why...| Negative|   300|
|So far 36 Pennsyl...| Negative|   265|
|Ignore online off...| Negative|   306|
+--------------------+---------+------+
only showing top 10 rows



In [230]:
data = data.na.drop()
data.show()

+--------------------+--------------------+------+
|       OriginalTweet|           Sentiment|length|
+--------------------+--------------------+------+
|TRENDING: New Yor...|  Extremely Negative|   228|
|When I couldn't f...|            Positive|   193|
|Find out how you ...|  Extremely Positive|    73|
|#Panic buying hit...|            Negative|   313|
|#toiletpaper #dun...|             Neutral|   252|
|Do you remember t...|             Neutral|   205|
|Voting in the age...|            Positive|    90|
|HI TWITTER! I am ...|  Extremely Negative|   280|
|Anyone been in a ...|  Extremely Positive|   238|
|Best quality couc...|            Positive|   224|
|Beware of counter...|  Extremely Negative|   240|
|"Panic food buyin...| taken from the w...|   179|
|#Covid_19 Went to...|  Extremely Positive|   280|
|While we were bus...|            Positive|   197|
|#AirSewa 

@fly...|  Extremely Negative|   287|
|What Precautionar...|  Extremely Positive|   246|
|When youre stock...|           

In [231]:
data = data.filter("Sentiment == 'Neutral' OR Sentiment == 'Negative' OR Sentiment == 'Positive' OR Sentiment == 'Extremely Negative' OR Sentiment == 'Extremely Positive'")
data.show()

+--------------------+------------------+------+
|       OriginalTweet|         Sentiment|length|
+--------------------+------------------+------+
|TRENDING: New Yor...|Extremely Negative|   228|
|When I couldn't f...|          Positive|   193|
|Find out how you ...|Extremely Positive|    73|
|#Panic buying hit...|          Negative|   313|
|#toiletpaper #dun...|           Neutral|   252|
|Do you remember t...|           Neutral|   205|
|Voting in the age...|          Positive|    90|
|HI TWITTER! I am ...|Extremely Negative|   280|
|Anyone been in a ...|Extremely Positive|   238|
|Best quality couc...|          Positive|   224|
|Beware of counter...|Extremely Negative|   240|
|#Covid_19 Went to...|Extremely Positive|   280|
|While we were bus...|          Positive|   197|
|#AirSewa 

@fly...|Extremely Negative|   287|
|What Precautionar...|Extremely Positive|   246|
|When youre stock...|           Neutral|   187|
|That's about a we...|          Positive|   239|
|Studies show the ...|

In [232]:
data = data.withColumn('Sentiment', regexp_replace('Sentiment', 'Extremely Negative', 'Negative'))
data = data.withColumn('Sentiment', regexp_replace('Sentiment', 'Extremely Positive', 'Positive'))

In [233]:
data.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|TRENDING: New Yor...| Negative|   228|
|When I couldn't f...| Positive|   193|
|Find out how you ...| Positive|    73|
|#Panic buying hit...| Negative|   313|
|#toiletpaper #dun...|  Neutral|   252|
|Do you remember t...|  Neutral|   205|
|Voting in the age...| Positive|    90|
|HI TWITTER! I am ...| Negative|   280|
|Anyone been in a ...| Positive|   238|
|Best quality couc...| Positive|   224|
|Beware of counter...| Negative|   240|
|#Covid_19 Went to...| Positive|   280|
|While we were bus...| Positive|   197|
|#AirSewa 

@fly...| Negative|   287|
|What Precautionar...| Positive|   246|
|When youre stock...|  Neutral|   187|
|That's about a we...| Positive|   239|
|Studies show the ...| Positive|   272|
|#CoronaVirus #COV...| Negative|   284|
|"For those of you...| Positive|   279|
+--------------------+---------+------+
only showing top 20 rows



# Features Transformation

In [234]:
tokenizer=Tokenizer(inputCol="OriginalTweet", outputCol="token_text")
stopremove=StopWordsRemover(inputCol="token_text", outputCol="stop_tokens")
count_vec=CountVectorizer(inputCol="stop_tokens", outputCol="c_vec")
#hashingTF = HashingTF(inputCol="stop_tokens", outputCol="c_vec", numFeatures=5000)
idf=IDF(inputCol="c_vec", outputCol="tf_idf")
# we also need to convert our labels in numbers
ham_samp_to_num = StringIndexer(inputCol="Sentiment", outputCol='label')
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

# Model

In [235]:
dtc=DecisionTreeClassifier(maxDepth=30)
rf=RandomForestClassifier(numTrees=200)
nb=NaiveBayes()

# Pipeline

In [236]:
data_prep_pipeline= Pipeline(stages=[ham_samp_to_num, tokenizer, stopremove,count_vec, idf,clean_up, nb])

In [237]:
data.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|TRENDING: New Yor...| Negative|   228|
|When I couldn't f...| Positive|   193|
|Find out how you ...| Positive|    73|
|#Panic buying hit...| Negative|   313|
|#toiletpaper #dun...|  Neutral|   252|
|Do you remember t...|  Neutral|   205|
|Voting in the age...| Positive|    90|
|HI TWITTER! I am ...| Negative|   280|
|Anyone been in a ...| Positive|   238|
|Best quality couc...| Positive|   224|
|Beware of counter...| Negative|   240|
|#Covid_19 Went to...| Positive|   280|
|While we were bus...| Positive|   197|
|#AirSewa 

@fly...| Negative|   287|
|What Precautionar...| Positive|   246|
|When youre stock...|  Neutral|   187|
|That's about a we...| Positive|   239|
|Studies show the ...| Positive|   272|
|#CoronaVirus #COV...| Negative|   284|
|"For those of you...| Positive|   279|
+--------------------+---------+------+
only showing top 20 rows



In [238]:
(training, testing)= data.randomSplit([0.8,0.2])

In [239]:
training.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
| Food poverty doe...| Negative|   225|
| There is enough ...|  Neutral|    97|
| This is the best...| Positive|   181|
|"""Current estima...|  Neutral|   138|
|"#BREAKING: Ontar...| Negative|   250|
|"#Covid_19 #Hamst...| Positive|   207|
|"?? Starbucks may...| Positive|   127|
|"@ABC7 #abc7eyewi...| Negative|   258|
|"@AskTarget ""cor...| Positive|   244|
|"@Ryanair is at i...| Positive|   259|
|"@colemank83 offe...| Positive|   221|
|"@iamshafaatali I...| Negative|   259|
|"@phineashead @Bo...| Negative|   259|
|"@quietplacemovie...| Positive|   245|
|"Amazon blocks th...| Negative|   251|
|"Apparently Chest...| Positive|   174|
|"Are we still all...| Positive|   168|
|"BJP MP Subramani...|  Neutral|   164|
|"Be like my girlf...| Positive|   277|
|"COVID-19 UPDATE:...| Positive|   241|
+--------------------+---------+------+
only showing top 20 rows



In [240]:
testing.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|  We ve got sever...| Negative|   235|
| Amazon Needs to ...| Negative|   203|
|!!!!!!! French Pr...| Negative|   133|
|"#Airbnb some hos...| Negative|   287|
|"2 Weeks #Lockdow...|  Neutral|   307|
|"@yogagirlmab Ano...| Negative|   261|
|"A ""consumer rem...| Positive|   259|
|"A friendly remin...| Positive|   271|
|"For those of you...| Positive|   279|
|"Our latest repor...| Negative|   178|
|#Americans stock ...|  Neutral|    94|
|#BBCqt @MattHanco...|  Neutral|    70|
|#COVID 19 VIRUS. ...| Negative|   280|
|#COVID19 is creat...| Negative|   307|
|#COVID19 pandemic...|  Neutral|   186|
|#COVID_19 fallout...| Negative|   206|
|#Colombo looks li...| Positive|   241|
|#Coronavirus #ori...| Positive|   262|
|#Coronavirus got ...| Negative|   277|
|#Coronavirus shif...|  Neutral|   106|
+--------------------+---------+------+
only showing top 20 rows



In [241]:
%%time
model = data_prep_pipeline.fit(training)

CPU times: user 42.1 ms, sys: 14 ms, total: 56.1 ms
Wall time: 3.26 s


In [242]:
model

PipelineModel_7f7a7604dcdb

### Naive Bayes

In [243]:
dx = spark.createDataFrame(testing.take(2))

In [244]:
dx.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|  We ve got sever...| Negative|   235|
| Amazon Needs to ...| Negative|   203|
+--------------------+---------+------+



In [245]:
test_results=model.transform(testing)

In [246]:
test_results.show()

+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|       OriginalTweet|Sentiment|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  We ve got sever...| Negative|   235|  0.0|[, , we, ve, got,...|[, , ve, got, sev...|(16792,[0,3,25,55...|(16792,[0,3,25,55...|(16793,[0,3,25,55...|[-1138.8170530833...|[0.99999999999898...|       0.0|
| Amazon Needs to ...| Negative|   203|  0.0|[, amazon, needs,...|[, amazon, needs,...|(16792,[0,7,9,10,...|(16792,[0,7,9,10,...|(16793,[0,7,9,10,...|[-1180.2882613284...|[5.1948749207

In [247]:
acc_eval=MulticlassClassificationEvaluator()
acc=acc_eval.evaluate(test_results)

In [248]:
print ("Accuracy of the model is::", acc)

Accuracy of the model is:: 0.5588572411186848


### Decision Tree Classifier

In [249]:
data_prep_pipeline= Pipeline(stages=[ham_samp_to_num, tokenizer, stopremove,count_vec, idf,clean_up, dtc])

In [250]:
model = data_prep_pipeline.fit(training)
model

PipelineModel_3bf58d3d6ce4

In [251]:
dx = spark.createDataFrame(testing.take(2))
dx.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|  We ve got sever...| Negative|   235|
| Amazon Needs to ...| Negative|   203|
+--------------------+---------+------+



In [252]:
test_results=model.transform(testing)
test_results.show()

+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+----------+
|       OriginalTweet|Sentiment|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|     rawPrediction|         probability|prediction|
+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+----------+
|  We ve got sever...| Negative|   235|  0.0|[, , we, ve, got,...|[, , ve, got, sev...|(16792,[0,3,25,55...|(16792,[0,3,25,55...|(16793,[0,3,25,55...|[382.0,249.0,88.0]|[0.53129346314325...|       0.0|
| Amazon Needs to ...| Negative|   203|  0.0|[, amazon, needs,...|[, amazon, needs,...|(16792,[0,7,9,10,...|(16792,[0,7,9,10,...|(16793,[0,7,9,10,...|[168.0,149.0,58.0]|[0.448,0.39733333...|  

In [253]:
acc_eval=MulticlassClassificationEvaluator()
acc=acc_eval.evaluate(test_results)

In [254]:
print ("Accuracy of the model is::", acc)

Accuracy of the model is:: 0.5810550423271459


### Random Forest

In [255]:
data_prep_pipeline= Pipeline(stages=[ham_samp_to_num, tokenizer, stopremove,count_vec, idf,clean_up, rf])

In [256]:
model = data_prep_pipeline.fit(training)
model

PipelineModel_4ef0d407e841

In [257]:
dx = spark.createDataFrame(testing.take(2))
dx.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|  We ve got sever...| Negative|   235|
| Amazon Needs to ...| Negative|   203|
+--------------------+---------+------+



In [258]:
test_results=model.transform(testing)

In [259]:
test_results.show()

+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|       OriginalTweet|Sentiment|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  We ve got sever...| Negative|   235|  0.0|[, , we, ve, got,...|[, , ve, got, sev...|(16792,[0,3,25,55...|(16792,[0,3,25,55...|(16793,[0,3,25,55...|[89.1184397689732...|[0.44559219884486...|       0.0|
| Amazon Needs to ...| Negative|   203|  0.0|[, amazon, needs,...|[, amazon, needs,...|(16792,[0,7,9,10,...|(16792,[0,7,9,10,...|(16793,[0,7,9,10,...|[84.1899843213239...|[0.4209499216

In [260]:
acc_eval=MulticlassClassificationEvaluator()
acc=acc_eval.evaluate(test_results)

In [261]:
print ("Accuracy of the model is::", acc)

Accuracy of the model is:: 0.4920932857411593
